### Домашняя работа №1.1 (Pandas) - максимум 3 балла
В городе SQL произошло убийство! SQL Murder Mystery - это одновременно и самостоятельный урок для изучения концепций и команд SQL, и увлекательная игра для опытных пользователей SQL, в которой нужно раскрыть интригующее преступление.
Произошло преступление, и детективу нужна ваша помощь. Детектив дал вам отчет о месте преступления, но вы каким-то образом потеряли его. Вы смутно помните, что преступление было убийством, произошедшим 15 января 2018 года, и что оно произошло в SQL City. Начните с поиска соответствующего отчета о месте преступления в базе данных полицейского управления.

### Условия
Главное условие - решение полностью с использованием Pandas

Данные не на русском языке, так что где-то придется пользоваться переводчиком

### Критерии оценивания

1. Нашел преступника? - 4 балла
2. Использование сортировки или срезов по данным (оператор []) во время поиска - 1 балл
3. Использование фильтрации таблиц - 2 балла
4. Использование мерджей - 3 балла


In [53]:
import pandas as pd
import os

data_dir = 'data_pandas'

crime_scene_report = pd.read_csv(os.path.join(data_dir, 'crime_scene_report.csv'))
person = pd.read_csv(os.path.join(data_dir, 'person.csv'))
interview = pd.read_csv(os.path.join(data_dir, 'interview.csv'))
drivers_license = pd.read_csv(os.path.join(data_dir, 'drivers_license.csv'))
get_fit_now_member = pd.read_csv(os.path.join(data_dir, 'get_fit_now_member.csv'))
get_fit_now_check_in = pd.read_csv(os.path.join(data_dir, 'get_fit_now_check_in.csv'))
facebook_event_checkin = pd.read_csv(os.path.join(data_dir, 'facebook_event_checkin.csv'))
income = pd.read_csv(os.path.join(data_dir, 'income.csv'))

print("Данные загружены")
print(f"Crime scene report: {len(crime_scene_report)} записей")
print(f"Person: {len(person)} записей")


Данные загружены
Crime scene report: 1228 записей
Person: 10011 записей


In [54]:
def find_crime_report(df, date, crime_type, city):
    """
    Находит отчет о преступлении по дате, типу и городу.
    Использует фильтрацию для поиска нужной записи.
    """
    mask = (df['date'] == date) & (df['type'] == crime_type) & (df['city'] == city)
    return df[mask]

crime_report = find_crime_report(crime_scene_report, 20180115, 'murder', 'SQL City')
print("Отчет о преступлении:")
print(crime_report[['date', 'type', 'description', 'city']])


Отчет о преступлении:
          date    type                                        description  \
1227  20180115  murder  Security footage shows that there were 2 witne...   

          city  
1227  SQL City  


In [55]:
def find_witness_on_street(df, street_name):
    """
    Находит первого свидетеля, который живет в последнем доме на указанной улице.
    Использует фильтрацию и сортировку для поиска последнего дома.
    """
    street_mask = df['address_street_name'] == street_name
    street_residents = df[street_mask].copy()
    street_residents = street_residents.sort_values('address_number', ascending=False)
    return street_residents.iloc[0] if len(street_residents) > 0 else None

def find_witness_by_name(df, name, street_name):
    """
    Находит второго свидетеля по имени и улице.
    Использует фильтрацию для поиска нужного человека.
    """
    mask = (df['name'].str.contains(name, case=False, na=False)) & \
           (df['address_street_name'] == street_name)
    witnesses = df[mask]
    return witnesses.iloc[0] if len(witnesses) > 0 else None

witness1 = find_witness_on_street(person, 'Northwestern Dr')
witness2 = find_witness_by_name(person, 'Annabel', 'Franklin Ave')

print(f"\nПервый свидетель: {witness1['name']}, ID: {witness1['id']}")
print(f"Второй свидетель: {witness2['name']}, ID: {witness2['id']}")



Первый свидетель: Morty Schapiro, ID: 14887
Второй свидетель: Annabel Miller, ID: 16371


In [56]:
def get_witness_interviews(interview_df, witness_ids):
    """
    Получает интервью свидетелей по их ID.
    Использует фильтрацию для поиска нужных интервью.
    """
    mask = interview_df['person_id'].isin(witness_ids)
    return interview_df[mask]

witness_ids = [witness1['id'], witness2['id']]
witness_interviews = get_witness_interviews(interview, witness_ids)
print("Интервью со свидетелями:")
for idx, row in witness_interviews.iterrows():
    print(f"\nСвидетель ID {row['person_id']}:")
    print(row['transcript'])


Интервью со свидетелями:

Свидетель ID 14887:
I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".

Свидетель ID 16371:
I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


In [57]:
def extract_clues_from_interviews(interviews_df):
    """
    Извлекает ключевые подсказки из интервью свидетелей.
    Использует строковые операции для поиска информации в тексте.
    """
    clues = {}
    for idx, row in interviews_df.iterrows():
        transcript = str(row['transcript']).lower()
        if '48z' in transcript:
            clues['membership_prefix'] = '48Z'
            clues['membership_status'] = 'gold'
        if 'h42w' in transcript:
            clues['plate_contains'] = 'H42W'
    return clues

clues = extract_clues_from_interviews(witness_interviews)
print(f"\nИзвлеченные подсказки: {clues}")

def find_suspect_by_gym_info(member_df, check_in_df, license_df, person_df, membership_prefix, status, plate_contains, gym_date):
    """
    Находит подозреваемого на основе информации о спортзале и машине.
    Использует множественные мерджи для объединения таблиц и фильтрацию по критериям.
    """
    gym_members = member_df[
        (member_df['id'].str.startswith(membership_prefix)) &
        (member_df['membership_status'] == status)
    ]
    
    if len(gym_members) == 0:
        return pd.DataFrame()
    
    gym_checkins = check_in_df[check_in_df['check_in_date'] == gym_date]
    merged_gym = gym_members.merge(gym_checkins, left_on='id', right_on='membership_id', how='inner')
    
    if len(merged_gym) == 0:
        return pd.DataFrame()
    
    person_gym = person_df.merge(merged_gym, left_on='id', right_on='person_id', how='inner', suffixes=('_person', '_gym'))
    
    if len(person_gym) == 0:
        return pd.DataFrame()
    
    person_license = person_gym.merge(license_df, left_on='license_id', right_on='id', how='inner', suffixes=('', '_license'))
    
    if len(person_license) == 0:
        return pd.DataFrame()
    
    final_mask = person_license['plate_number'].str.contains(plate_contains, case=False, na=False)
    return person_license[final_mask]

suspects = find_suspect_by_gym_info(
    get_fit_now_member, 
    get_fit_now_check_in,
    drivers_license,
    person,
    clues.get('membership_prefix', '48Z'),
    clues.get('membership_status', 'gold'),
    clues.get('plate_contains', 'H42W'),
    20180109
)

print(f"\nНайдено подозреваемых: {len(suspects)}")
if len(suspects) > 0:
    id_col = 'id_person' if 'id_person' in suspects.columns else 'id'
    name_col = 'name_person' if 'name_person' in suspects.columns else 'name'
    cols_to_show = [id_col, name_col, 'license_id', 'address_street_name', 'plate_number']
    cols_to_show = [col for col in cols_to_show if col in suspects.columns]
    print(suspects[cols_to_show])



Извлеченные подсказки: {'membership_prefix': '48Z', 'membership_status': 'gold', 'plate_contains': 'H42W'}

Найдено подозреваемых: 1
   id_person    name_person  license_id    address_street_name plate_number
0      67318  Jeremy Bowers      423327  Washington Pl, Apt 3A       0H42W2


In [58]:
def verify_killer_at_crime_scene(member_df, check_in_df, suspect_id, crime_date):
    """
    Проверяет, был ли подозреваемый в спортзале в день преступления.
    Использует мердж и фильтрацию для проверки наличия чекина в указанную дату.
    """
    member_info = member_df[member_df['person_id'] == suspect_id]
    if len(member_info) == 0:
        return False
    
    membership_id = member_info.iloc[0]['id']
    merged = check_in_df.merge(member_info, left_on='membership_id', right_on='id', how='inner')
    date_mask = merged['check_in_date'] == crime_date
    return len(merged[date_mask]) > 0

def find_killer(member_df, check_in_df, license_df, person_df, interview_df, clues, crime_date, gym_date):
    """
    Находит убийцу на основе всех собранных данных из интервью.
    Использует множественные мерджи и фильтрацию для идентификации преступника.
    Второй свидетель видел убийцу в спортзале 9 января, что используется для идентификации.
    """
    suspects = find_suspect_by_gym_info(
        member_df, check_in_df, license_df, person_df,
        clues.get('membership_prefix', '48Z'),
        clues.get('membership_status', 'gold'),
        clues.get('plate_contains', 'H42W'),
        gym_date
    )
    
    if len(suspects) == 0:
        return None
    
    for idx, suspect in suspects.iterrows():
        person_id = suspect.get('id_person', suspect.get('id'))
        suspect_interview = interview_df[interview_df['person_id'] == person_id]
        return {
            'person': suspect,
            'interview': suspect_interview.iloc[0]['transcript'] if len(suspect_interview) > 0 else None
        }
    
    return None

killer_info = find_killer(
    get_fit_now_member, 
    get_fit_now_check_in,
    drivers_license,
    person,
    interview,
    clues,
    20180115,
    20180109
)

if killer_info is not None:
    killer = killer_info['person']
    person_id = killer.get('id_person', killer.get('id'))
    person_name = killer.get('name_person', killer.get('name'))
    address_num = killer.get('address_number', 'N/A')
    address_street = killer.get('address_street_name', 'N/A')
    ssn = killer.get('ssn', 'N/A')
    
    print("\n" + "="*50)
    print("НАЙДЕН ПРЕСТУПНИК!")
    print("="*50)
    print(f"Имя: {person_name}")
    print(f"ID: {int(person_id)}")
    print(f"Адрес: {int(address_num)} {address_street}")
    print(f"SSN: {int(ssn)}")
    if killer_info['interview']:
        print(f"\nИнтервью: {killer_info['interview']}")
else:
    print("\nПреступник не найден")



НАЙДЕН ПРЕСТУПНИК!
Имя: Jeremy Bowers
ID: 67318
Адрес: 530 Washington Pl, Apt 3A
SSN: 871539279

Интервью: I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.



## Логика работы решения

### Шаг 1: Загрузка данных (Cell 3)
Все данные загружаются в pandas DataFrame с помощью `pd.read_csv()`. Это полностью Pandas операция.

### Шаг 2: Поиск отчета о преступлении (Cell 4)
**Используется фильтрация (критерий 3):**
- Создается булева маска через условия: `(df['date'] == date) & (df['type'] == crime_type) & (df['city'] == city)`
- Применяется фильтрация через оператор `[]`: `df[mask]`
- Из отчета извлекается информация о двух свидетелях

### Шаг 3: Поиск свидетелей (Cell 5)
**Используется фильтрация и сортировка/срезы (критерии 2 и 3):**

**Первый свидетель:**
- Фильтрация по улице: `df['address_street_name'] == street_name`
- Сортировка по номеру дома: `sort_values('address_number', ascending=False)` - находит последний дом
- Срез через `iloc[0]` для получения первой записи (после сортировки это последний дом)

**Второй свидетель:**
- Фильтрация по имени (содержит "Annabel") и улице: `df['name'].str.contains(name) & (df['address_street_name'] == street_name)`
- Срез через `iloc[0]` для получения результата

### Шаг 4: Получение интервью (Cell 6)
**Используется фильтрация (критерий 3):**
- Фильтрация через `isin()`: `interview_df['person_id'].isin(witness_ids)`
- Получаем текстовые интервью со свидетелей

### Шаг 5: Извлечение подсказок (Cell 7)
Анализ текста интервью для извлечения ключевых данных:
- Первый свидетель: номер членства начинается с "48Z", статус "gold", номер машины содержит "H42W"
- Второй свидетель: видел убийцу в спортзале 9 января 2018

### Шаг 6: Поиск подозреваемого (Cell 7)
**Используются множественные мерджи и фильтрация (критерии 3 и 4):**

1. **Фильтрация членов спортзала:**
   - `member_df['id'].str.startswith('48Z')` - фильтрация по префиксу
   - `member_df['membership_status'] == 'gold'` - фильтрация по статусу

2. **Фильтрация чекинов:**
   - `check_in_df['check_in_date'] == gym_date` - фильтрация по дате (9 января)

3. **Мердж 1:** `gym_members.merge(gym_checkins, ...)` - объединение членов и чекинов
   - Находим кто из gold-членов с префиксом "48Z" был в спортзале 9 января

4. **Мердж 2:** `person_df.merge(merged_gym, ...)` - объединение с таблицей людей
   - Получаем информацию о людях (имя, адрес, license_id)

5. **Мердж 3:** `person_gym.merge(license_df, ...)` - объединение с водительскими правами
   - Получаем информацию о машине (номерной знак)

6. **Финальная фильтрация:**
   - `person_license['plate_number'].str.contains('H42W', ...)` - фильтрация по номеру машины

### Шаг 7: Идентификация убийцы (Cell 8)
**Используется фильтрация (критерий 3):**
- Получение интервью подозреваемого: `interview_df[interview_df['person_id'] == person_id]`
- Вывод информации о преступнике